# Airbnb Price Prediction — Pipeline Modeling

Goal:
Build an end-to-end machine learning pipeline that handles
preprocessing and modeling correctly using scikit-learn.


In [133]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [118]:
df = pd.read_csv("../data/raw/airbnb.csv",encoding='latin1',low_memory=False)
df.head()


,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


In [135]:
y = np.log1p(df["price"])


In [145]:
features = [
    "accommodates",
    "bedrooms",
    "latitude",            # Precise location (Numerical)
    "longitude",           # Precise location (Numerical)
    "room_type",           # Property category
    "city",                # Broad location
    "review_scores_rating" # Quality metric
]
X = df[features]

In [148]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Split Before Preprocessing: It prevents "data leakage" by ensuring the model never sees any information (like means or scales) from the test set during training.

In [147]:
num=X.select_dtypes(include=['float64','int64']).columns
cat=X.select_dtypes(include=['object']).columns

In [149]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="constant")),("scaler", StandardScaler())])
cat_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="constant")),("encoder", OneHotEncoder(handle_unknown="ignore"))])

In [150]:
preprocessor=ColumnTransformer(transformers=[('numer',num_pipeline,num),('categ',cat_pipeline,cat)])


Preprocessing inside Pipeline:
 It automates the "fit" and "transform" steps so that data cleaning happens as a single, unified operation during model training and prediction.

Preventing Data Leakage:
 keeping the test set completely unseen and realistic

In [151]:
linreg_pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("model", LinearRegression())
])


In [152]:
linreg_pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('numer', ...), ('categ', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

In [153]:
y_pred = linreg_pipeline.predict(X_test)


In [154]:
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, rmse, r2


(0.46296668704140903, 0.41053307721560534, 0.743661632393761)

**RMSE (Log Scale):**  an RMSE of 0.3 indicates predictions are typically off by about 30%.
**$R^2$ Score:** Shows the percentage of price variation explained by the model; a score of 0.79 means the model accounts for about 80% of price differences.
**Limitation:** While these numbers track performance, they don't reveal if the model is specifically failing on very high-end or very low-end Airbnb listings.

## Random Forest Pipeline

In [159]:
model = RandomForestRegressor(
    n_estimators=200,      # More trees for a stable R2
    max_depth=20,          # Deeper trees to capture neighborhood price pockets
    max_features='sqrt',   # Keeps it fast by looking at a subset of features per split
    min_samples_leaf=5,    # Prevents overfitting to single-listing outliers
    random_state=42,
    n_jobs=-1              # Uses all your CPU cores
)


In [163]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])

In [156]:
# Check unique values in each categorical column
for col in cat:
    print(f"{col}: {X_train[col].nunique()} unique values")

room_type: 4 unique values
city: 10 unique values


In [164]:
rf_pipeline.fit(X_train, y_train)

rf_pred = rf_pipeline.predict(X_test)

In [165]:

rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

rf_mae, rf_rmse, rf_r2


(0.384575983975758, 0.2959689854574729, 0.8151958738408829)

##  Model Performance Summary

After applying proper preprocessing pipelines and training a Random Forest model on a log-transformed target variable, the final model achieved strong performance:

- **R² Score:** 0.82
- **RMSE (log scale):** 0.30

### Key Takeaways
- Log transformation significantly improved model stability and reduced the impact of extreme prices.
- Tree-based models captured non-linear relationships that linear regression could not.
- Using scikit-learn pipelines ensured correct preprocessing and prevented data leakage.


